# Alpha Diversity Analysis

We will first test for associations between our categorical metadata columns and alpha diversity. Alpha diversity asks about the distribution of features within each sample, and once calculated for all samples can be used to test whether the per‐sample diversity differs across different conditions (e.g., samples obtained at different ages). The comparison makes no assumptions about the features that are shared between samples; two samples can have the same alpha diversity and not share any features.

In [11]:
# Import libraries
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import numpy as np

%matplotlib inline

In [12]:
data_dir = 'data'
data_dir_phyl = 'data/phylogeny' # used when attemping phylogentic based analysis
data_dir_div = 'data/alpha_diversity'

## Alpha Rarefaction

To perform rarefaction, we first need to decide which sampling depth is best suited for our dataset. For this, we will analyse how sampling depth impacts within-sample diversity estimates (= alpha diversity) with the alpha-rarefaction action. This action generates interactive alpha rarefaction curves for sequencing depths between min_depth and max_depth and computes 10 (default) rarefied tables with corresponding alpha diversity metrics at each sampling depth step.

In [13]:
# Generates alpha rarefaction curves
! qiime diversity alpha-rarefaction \
    --i-table $data_dir/closed_reference_cluster/table-filtered.qza \
    --p-max-depth 10000 \
    --m-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir_div/alpha_rarefaction.qzv

#   --i-phylogeny $data_dir_phyl/fasttree_tree_rooted.qza \

^C

Aborted!


In [4]:
Visualization.load(f"{data_dir_div}/alpha_rarefaction.qzv")

<visualization: Visualization uuid: 7e0a7e63-e521-432c-b65c-c94e8d10291b>

## Diversity Analysis
Applies a collection of diversity metrics (non-phylogenetic) to a feature
table. For alpha diversity three metrics are important: 
- Shannon Entropy: a quantitative measure of community richness
- Pielou Evenness: a measure of community evenness
- observed features: a quantitative measure of community richness, called “observed OTUs” here for historical reasons

In [5]:
! qiime diversity core-metrics \
  --i-table $data_dir/closed_reference_cluster/table-filtered.qza \
  --m-metadata-file $data_dir/pundemic_metadata.tsv \
  --p-sampling-depth 3000 \
  --p-n-jobs 8 \
  --output-dir $data_dir_div/core_metrics_results

Saved FeatureTable[Frequency] to: data/alpha_diversity/core_metrics_results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: data/alpha_diversity/core_metrics_results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: data/alpha_diversity/core_metrics_results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: data/alpha_diversity/core_metrics_results/evenness_vector.qza
Saved DistanceMatrix to: data/alpha_diversity/core_metrics_results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: data/alpha_diversity/core_metrics_results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: data/alpha_diversity/core_metrics_results/jaccard_pcoa_results.qza
Saved PCoAResults to: data/alpha_diversity/core_metrics_results/bray_curtis_pcoa_results.qza
Saved Visualization to: data/alpha_diversity/core_metrics_results/jaccard_emperor.qzv
Saved Visualization to: data/alpha_diversity/core_metrics_results/bray_curtis_emperor.qzv


In [14]:
metadata = pd.read_csv(f"{data_dir}/pundemic_metadata.tsv", sep='\t')

## Pairwise difference comparisons between time points
When microbial data are collected at different timepoints, it is useful to examine dynamic changes in the microbial communities (longitudinal analysis). This Pairwise difference test determines whether the value of a specific metric changed significantly between pairs of paired samples (e.g., pre- and post-treatment).

All donors were classified under ”unknown” sex while patients had known sexes, this effect is likely unrelated to biological sex differences. To investigate further, samples with ”unknown” sex were excluded. The existing categories for disease_status did not differentiate
between healed and unaffected patients post-FMT treatment, limiting interpretability. To address this, a
new metadata category was introduced, distinguishing between healthy, healed, and diseased patients.

To enhance the evaluation of treatment response, a new metadata category, subgroup response, was
introduced, which combined treatment group (placebo or FMT) with clinical response (response or no response). However, rarefaction filtering resulted in the exclusion of the post-treatment sample from the
only placebo patient who responded to treatment, necessitating its removal from subsequent analyses.

In [22]:
metadata['subgroup_response'] = metadata['disease_subgroup'] + "_" + metadata['blinded_clinical_response']

# Change all Unknown with NAs so it does not occur in the plots as separate box
metadata.age = metadata.age.replace('Unknown', np.nan)
metadata.sex = metadata.sex.replace('Unknown', np.nan)

# Differentiate healed and healthy and check for significance between healthy, healed, disease
metadata['disease_status'] = metadata['group']
metadata.loc[(metadata.time_point == 'post-treatment') & (metadata.blinded_clinical_response == 'Res'), 'disease_status'] = 'Healed'

# Differentiate subgroup response groups between time points
metadata['subgroup_response_time'] = metadata['subgroup_response'] + "_" + metadata['time_point']
metadata.to_csv(f'{data_dir}/pundemic_metadata_subgroup_response_all.tsv', sep='\t', index=False)

In [8]:
! qiime tools export \
  --input-path $data_dir_div/core_metrics_results/shannon_vector.qza \
  --output-path $data_dir_div/shannon

Exported data/alpha_diversity/core_metrics_results/shannon_vector.qza as AlphaDiversityDirectoryFormat to directory data/alpha_diversity/shannon


In [9]:
! qiime tools export \
  --input-path $data_dir_div/core_metrics_results/evenness_vector.qza \
  --output-path $data_dir_div/evenness

Exported data/alpha_diversity/core_metrics_results/evenness_vector.qza as AlphaDiversityDirectoryFormat to directory data/alpha_diversity/evenness


In [10]:
! qiime tools export \
  --input-path $data_dir_div/core_metrics_results/observed_features_vector.qza \
  --output-path $data_dir_div/observed_features

Exported data/alpha_diversity/core_metrics_results/observed_features_vector.qza as AlphaDiversityDirectoryFormat to directory data/alpha_diversity/observed_features


In [11]:
shannon = pd.read_csv(f"{data_dir_div}/shannon/alpha-diversity.tsv", sep='\t')
evenness = pd.read_csv(f"{data_dir_div}/evenness/alpha-diversity.tsv", sep='\t')
observed_features = pd.read_csv(f"{data_dir_div}/observed_features/alpha-diversity.tsv", sep='\t')

In [12]:
shannon.rename(columns = {shannon.columns[0]: "id"}, inplace = True)
evenness.rename(columns = {evenness.columns[0]: "id"}, inplace = True)
observed_features.rename(columns = {observed_features.columns[0]: "id"}, inplace = True)

In [13]:
metrics = pd.merge(shannon, evenness, on = "id")
metrics = pd.merge(metrics, observed_features, on = "id")
metrics.head()

,id,shannon_entropy,pielou_evenness,observed_features
0,SRR10505051,1.317129,0.396495,10
1,SRR10505052,0.919479,0.224951,17
2,SRR10505053,0.231316,0.062510,13
3,SRR10505056,1.892840,0.453927,18
4,SRR10505057,1.562509,0.492917,9


In [14]:
metadata = pd.merge(metadata, metrics, on = "id")
metadata.head()

,id,patient_id,age,sex,ethnicity,continent,country,region,city,group,disease_subgroup,blinded_clinical_response,puns_per_hour_pre_treatment,puns_per_hour_post_treatment,time_point,subgroup_response,disease_status,shannon_entropy,pielou_evenness,observed_features
0,SRR10505051,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NR,9.0,8.0,post-treatment,Placebo_NR,Puns,1.317129,0.396495,10
1,SRR10505052,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NR,9.0,8.0,pre-treatment,Placebo_NR,Puns,0.919479,0.224951,17
2,SRR10505053,1045,29,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,Res,6.0,0.0,pre-treatment,Placebo_Res,Puns,0.231316,0.062510,13
3,SRR10505056,1044,34,male,Indian Subcontinental,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NaN,4.0,NaN,post-treatment,NaN,Puns,1.892840,0.453927,18
4,SRR10505057,1043,35,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,FMT,NR,9.0,6.0,post-treatment,FMT_NR,Puns,1.562509,0.492917,9


In [16]:
# Account for sex bias
metadata['shannon_sex_bias'] = metadata['shannon_entropy']

mean_female = np.mean(metadata.loc[metadata.sex == 'female', 'shannon_sex_bias'])
std_female = np.std(metadata.loc[metadata.sex == 'female', 'shannon_sex_bias'])

mean_male = np.mean(metadata.loc[metadata.sex == 'male', 'shannon_sex_bias'])
std_male = np.std(metadata.loc[metadata.sex == 'male', 'shannon_sex_bias'])

#normalization: z-scores for males and females separately
metadata.loc[metadata.sex == 'female', 'shannon_sex_bias'] = (metadata.loc[metadata.sex == 'female', 'shannon_sex_bias'] - mean_female)/std_female
metadata.loc[metadata.sex == 'male', 'shannon_sex_bias'] = (metadata.loc[metadata.sex == 'male', 'shannon_sex_bias'] - mean_male)/std_male

metadata.head()

,id,patient_id,age,sex,ethnicity,continent,country,region,city,group,...,blinded_clinical_response,puns_per_hour_pre_treatment,puns_per_hour_post_treatment,time_point,subgroup_response,disease_status,shannon_entropy,pielou_evenness,observed_features,shannon_sex_bias
0,SRR10505051,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,9.0,8.0,post-treatment,Placebo_NR,Puns,1.317129,0.396495,10,0.012471
1,SRR10505052,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,9.0,8.0,pre-treatment,Placebo_NR,Puns,0.919479,0.224951,17,-0.551280
2,SRR10505053,1045,29,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,Res,6.0,0.0,pre-treatment,Placebo_Res,Puns,0.231316,0.062510,13,-1.311910
3,SRR10505056,1044,34,male,Indian Subcontinental,Europe,Switzerland,Zurich,Zurich,Puns,...,NaN,4.0,NaN,post-treatment,NaN,Puns,1.892840,0.453927,18,1.183356
4,SRR10505057,1043,35,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,9.0,6.0,post-treatment,FMT_NR,Puns,1.562509,0.492917,9,0.360348


#### Note**

There was only one patient with a placebo response in our metadata, however the post-treatment sample got filtered out at the 3000 cut in the rarefaction step:

The state "post-treatment" is not represented by any members of Placebo_Res group in metadata. Filter out the "subgroup_response != "Placebo_Res""

In [17]:
metadata_filtered = metadata[metadata.subgroup_response.notna()]
metadata_filtered = metadata_filtered[metadata_filtered.subgroup_response != "Placebo_Res"]
metadata_filtered.to_csv(f'{data_dir}/pundemic_metadata_subgroup_response.tsv', sep='\t', index=False)
metadata_filtered

,id,patient_id,age,sex,ethnicity,continent,country,region,city,group,...,blinded_clinical_response,puns_per_hour_pre_treatment,puns_per_hour_post_treatment,time_point,subgroup_response,disease_status,shannon_entropy,pielou_evenness,observed_features,shannon_sex_bias
0,SRR10505051,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,9.0,8.0,post-treatment,Placebo_NR,Puns,1.317129,0.396495,10,0.012471
1,SRR10505052,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,9.0,8.0,pre-treatment,Placebo_NR,Puns,0.919479,0.224951,17,-0.551280
4,SRR10505057,1043,35,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,9.0,6.0,post-treatment,FMT_NR,Puns,1.562509,0.492917,9,0.360348
5,SRR10505058,1043,35,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,9.0,6.0,pre-treatment,FMT_NR,Puns,1.983531,0.444795,22,0.957233
7,SRR10505061,2212,27,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,8.0,4.0,pre-treatment,FMT_NR,Puns,1.050672,0.374257,7,-0.081407
8,SRR10505062,1041,39,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,Res,6.0,0.0,post-treatment,FMT_Res,Healed,1.285828,0.387073,10,-0.031904
9,SRR10505063,1041,39,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,Res,6.0,0.0,pre-treatment,FMT_Res,Puns,1.778664,0.393200,23,0.666792
10,SRR10505064,1040,52,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,7.0,7.0,pre-treatment,Placebo_NR,Puns,0.496077,0.213649,5,-0.914293
11,SRR10505065,1040,52,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,7.0,7.0,post-treatment,Placebo_NR,Puns,2.124552,0.441938,28,1.531339
12,SRR10505066,1038,35,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,...,NR,6.0,2.0,pre-treatment,FMT_NR,Puns,1.381490,0.314524,21,0.415413


### Shannon Entropy

In [18]:
# Compares Shannon entropy differences for pre and post
! qiime longitudinal pairwise-differences \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response.tsv \
  --p-metric shannon_entropy \
  --p-group-column subgroup_response \
  --p-state-column time_point \
  --p-state-1 pre-treatment \
  --p-state-2 post-treatment \
  --p-individual-id-column patient_id \
  --p-replicate-handling random \
  --o-visualization $data_dir_div/shannon-pairwise-differences.qzv

Saved Visualization to: data/alpha_diversity/shannon-pairwise-differences.qzv


In [8]:
Visualization.load(f"{data_dir_div}/shannon-pairwise-differences.qzv")

<visualization: Visualization uuid: 1c0c897d-cade-4e55-b064-9cc548bf2570>

In [20]:
# Account for sex bias
! qiime longitudinal pairwise-differences \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response.tsv \
  --p-metric shannon_sex_bias \
  --p-group-column subgroup_response \
  --p-state-column time_point \
  --p-state-1 pre-treatment \
  --p-state-2 post-treatment \
  --p-individual-id-column patient_id \
  --p-replicate-handling random \
  --o-visualization $data_dir_div/shannon-sex_bias-pairwise-differences.qzv

Saved Visualization to: data/alpha_diversity/shannon-sex_bias-pairwise-differences.qzv


In [4]:
Visualization.load(f"{data_dir_div}/shannon-sex_bias-pairwise-differences.qzv")

<visualization: Visualization uuid: de6b66b0-b3f3-471d-958d-4cae040bb147>

### Pielou Evenness

In [22]:
! qiime longitudinal pairwise-differences \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response.tsv \
  --p-metric pielou_evenness \
  --p-group-column subgroup_response \
  --p-state-column time_point \
  --p-state-1 pre-treatment \
  --p-state-2 post-treatment \
  --p-individual-id-column patient_id \
  --p-replicate-handling random \
  --o-visualization $data_dir_div/evenness-pairwise-differences.qzv

Saved Visualization to: data/alpha_diversity/evenness-pairwise-differences.qzv


In [5]:
Visualization.load(f"{data_dir_div}/evenness-pairwise-differences.qzv")

<visualization: Visualization uuid: 05c95151-d92b-4b72-8634-455f95ef98d9>

### Observed Features

In [24]:
! qiime longitudinal pairwise-differences \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response.tsv \
  --p-metric observed_features \
  --p-group-column subgroup_response \
  --p-state-column time_point \
  --p-state-1 pre-treatment \
  --p-state-2 post-treatment \
  --p-individual-id-column patient_id \
  --p-replicate-handling random \
  --o-visualization $data_dir_div/observed_features-pairwise-differences.qzv

Saved Visualization to: data/alpha_diversity/observed_features-pairwise-differences.qzv


In [6]:
Visualization.load(f"{data_dir_div}/observed_features-pairwise-differences.qzv")

<visualization: Visualization uuid: 5f507093-7674-4d43-bc5d-0e2f80f5c5f4>

#### Preliminary result:

There are no significant differences between the different groups ("Pairwise group comparison tests"). The same thing can be concluded from the Pairwise difference tests, which states that there are no significant differences between the pre- and post-treatment samples.

## Diversity differences between categorical metadata 
The rarefied SampleData[AlphaDiversity] artifact produced in the above step contains univariate, continuous values and can be tested using common non‐parametric statistical test (e.g., Kruskal‐Wallis test)

In [23]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir_div/core_metrics_results/shannon_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata.tsv \
  --o-visualization $data_dir_div/core_metrics_results/no_healed_shannon_vector.qzv

Saved Visualization to: data/alpha_diversity/core_metrics_results/no_healed_shannon_vector.qzv


In [24]:
Visualization.load(f"{data_dir_div}/core_metrics_results/no_healed_shannon_vector.qzv")

<visualization: Visualization uuid: e6fcf1f4-23c9-4ac9-b293-b4ae0acefc6c>

### Differentiating between Healthy, Healed and Puns

In [25]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir_div/core_metrics_results/shannon_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response_all.tsv \
  --o-visualization $data_dir_div/core_metrics_results/shannon_vector.qzv

Saved Visualization to: data/alpha_diversity/core_metrics_results/shannon_vector.qzv


In [26]:
Visualization.load(f"{data_dir_div}/core_metrics_results/shannon_vector.qzv")

<visualization: Visualization uuid: 81943122-6245-4eac-919b-6d5658f7a1bf>

In [30]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir_div/core_metrics_results/evenness_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response_all.tsv \
  --o-visualization $data_dir_div/core_metrics_results/evenness_vector.qzv

Saved Visualization to: data/alpha_diversity/core_metrics_results/evenness_vector.qzv


In [34]:
Visualization.load(f"{data_dir_div}/core_metrics_results/evenness_vector.qzv")

<visualization: Visualization uuid: ac6fa371-4ce0-4f35-914c-002e91da4e88>

In [32]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir_div/core_metrics_results/observed_features_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response_all.tsv \
  --o-visualization $data_dir_div/core_metrics_results/observed_features_vector.qzv

Saved Visualization to: data/alpha_diversity/core_metrics_results/observed_features_vector.qzv


In [35]:
Visualization.load(f"{data_dir_div}/core_metrics_results/observed_features_vector.qzv")

<visualization: Visualization uuid: 19642a27-ef23-4fae-89ef-7ffc06c3339d>

One important confounding factor here is that we are simultaneously analyzing our samples across all time points and in doing so potentially losing meaningful signals at a particular time point. Importantly, having more than one time point per subject also violates the assumption of the Kurskal‐Wallis test that all samples are independent. More appropriate methods that take into account repeated measurements from the same samples are demonstrated in the longitudinal paiwise data analysis section above.

## Alpha Correlation of numeric data (puns_per_hour)

In [36]:
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir_div/core_metrics_results/shannon_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response_all.tsv \
  --o-visualization $data_dir_div/shannon_alpha_correlation.qzv

Saved Visualization to: data/alpha_diversity/shannon_alpha_correlation.qzv


In [39]:
Visualization.load(f"{data_dir_div}/shannon_alpha_correlation.qzv")

<visualization: Visualization uuid: 0373fbf4-77bc-4231-a214-0557f1202018>

In [37]:
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir_div/core_metrics_results/evenness_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response_all.tsv \
  --o-visualization $data_dir_div/evenness_alpha_correlation.qzv

Saved Visualization to: data/alpha_diversity/evenness_alpha_correlation.qzv


In [40]:
Visualization.load(f"{data_dir_div}/evenness_alpha_correlation.qzv")

<visualization: Visualization uuid: dfa54cf4-9aa1-4499-aea3-9a4043c49301>

In [38]:
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir_div/core_metrics_results/observed_features_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata_subgroup_response_all.tsv \
  --o-visualization $data_dir_div/observed_features_alpha_correlation.qzv

Saved Visualization to: data/alpha_diversity/observed_features_alpha_correlation.qzv


In [41]:
Visualization.load(f"{data_dir_div}/observed_features_alpha_correlation.qzv")

<visualization: Visualization uuid: aa4552e4-9f79-47e5-8062-44ad1260c0d6>

# Conclusion

- significant difference between healthy and disease people BUT healed and healthy are seen as one group
    - add metadata columns: healed, healthy, diseased and do significance test
    - we saw a significant difference between healthy and puns but not healed and healthy
    - then pairwise testing: no significant difference
    
- We noticed a significant difference between sexes. This could lead to a bias in the analysis.
    - new metadata column for shannon with z-normalized by sex data
    - then pairwise testing: no significant difference 
    